In [2]:
from HkVp_multilayer.ac import AC 
import numpy as np 
import pandas as pd 
from obspy import read

In [3]:
##prepare data 

maindir = "seismic_data_RFAC"
excel_AC_para = pd.read_excel("AC_parameters.xlsx")
paras = excel_AC_para.iloc[:,:].values
indexes = excel_AC_para.index

for index in indexes:
 
        stationname = paras[index,0]
        if stationname != "TASTE_T03":
             continue
        datadir = f"{maindir}/{stationname}"
        print(index,stationname)
        ##for P-wave
        f1s_P = paras[index,1]
        f2s_P = paras[index,2]
        f1c_P = paras[index,7]
        f2c_P = paras[index,8]
        ##for S-wave
        f1s_S = paras[index,4]
        f2s_S = paras[index,5]
        f1c_S = paras[index,10]
        f2c_S = paras[index,11]
        ## for width of spectral whetening
        dfs_P = paras[index,3]
        dfs_S = paras[index,6]
        dfc_P = paras[index,9]
        dfc_S = paras[index,12]
        try:
             
            ##for P-wave reflected at the bottom of sediments
            do_Ac = AC(datadir=datadir,suffix=f".?HZ.sac",tref=5.,pretime=-5,length=100,f1=f1s_P,f2=f2s_P,tcos=0.5,df=dfs_P)
            do_Ac.cal_ACs()
            stream_zacs = do_Ac.ACst

            ##for S-wave reflected at the bottom of sediments
            do_Ac = AC(datadir=datadir,suffix=f".?HR.sac",tref=5.,pretime=-5,length=150,f1=f1s_S,f2=f2s_S,tcos=1,df=dfs_S)
            do_Ac.cal_ACs()
            stream_racs = do_Ac.ACst

            ##for P-wave reflected at Moho
            do_Ac = AC(datadir=datadir,suffix=f".?HZ.sac",tref=5.,pretime=-5,length=100,f1=f1c_P,f2=f2c_P,tcos=5,df=dfc_P)
            do_Ac.cal_ACs()
            stream_zacm = do_Ac.ACst

            ##for S-wave reflected at Moho
            do_Ac = AC(datadir=datadir,suffix=f".?HR.sac",tref=5.,pretime=-5,length=150,f1=f1c_S,f2=f2c_S,tcos=5,df=dfc_S)
            do_Ac.cal_ACs()
            stream_racm = do_Ac.ACst

            ##read RF data
            stream_rf2 = read(f"{datadir}/20*.?HR.sac_g2")
            stream_rf7 = read(f"{datadir}/20*.?HR.sac_g5")
        except:
             print("Pass")
             continue

        delta = 0.01
        pretimes = [5,5,0,0,0,0]
        length = 90
        npts = int(length/delta)
        norm_type = [1,1,-1,-1,-1,-1]
        streams = [stream_rf7,stream_rf2,stream_zacs,stream_zacm,stream_racs,stream_racm]
        ndataset = len(streams)
        data_stack = np.zeros([ndataset,len(stream_rf7),int(length/delta)],dtype=np.float32)
        ray_params = []
        for i in range(ndataset):
            ntrace = len(streams[i])
            pretime = pretimes[i]
            ray_params_temp =  []
            data_st = []
            for j in range(ntrace):
                starttime = streams[i][j].stats.starttime
                streams[i][j].resample(1/delta)
                prenpts = int(pretime/delta)
                data = streams[i][j].data[prenpts:prenpts+npts]
                data = data/(data.max()-data.min())
                data_st.append(data)
                ray_params_temp.append(streams[i][j].stats.sac['user0'])
            data_ray_zip = zip(ray_params_temp,data_st)
            data_ray_zip_sorted = sorted(data_ray_zip,key=lambda x:x[0])
            ray_params_temp,data_st = zip(*data_ray_zip_sorted)
            for j,ray_param in enumerate(ray_params_temp):
                if i==0:
                    ray_params.append(ray_param)
                data_stack[i,j,:] = data_st[j]

        # Save the data_stack to a numpy file
        np.savez(f"{datadir}/data_stack52.npz", data_stack=data_stack, ray_params=ray_params)
        print("finished")

32 TASTE_T03
finished
